<a href="https://colab.research.google.com/github/minhaz1172/NLP/blob/main/Sentiment_analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checking GPU and move to gpu

In [32]:
import torch

is_cuda = torch.cuda.is_available()

if is_cuda:
  device=torch.device('cuda')
  print("GPU Available")
else:
  device=torch.device('cpu')
  print("GPU NOT available")

print(f"Using device: {device}")

GPU Available
Using device: cuda


In [33]:
from google.colab import files
files.upload()  # Upload your kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minhaz1172","key":"c20c43ca9e6e5be6fa69c912fab503c3"}'}

In [34]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


# Download the IMDB Dataset via Kaggle API

In [35]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [36]:
!unzip imdb-dataset-of-50k-movie-reviews.zip -d data


Archive:  imdb-dataset-of-50k-movie-reviews.zip
replace data/IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [37]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# LowerCasing

In [38]:
df['review']=df['review'].str.lower()
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


# rEMOVE pUNCTUATION

In [39]:
import string
def removepunc(text):
  text=text.translate(str.maketrans('','',string.punctuation))
  return text

df['review']=df['review'].apply(lambda x:removepunc(x))

# rEMOVING sTOPWORDS

In [40]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Load the dataframe
#df = pd.read_csv('data/IMDB Dataset.csv')

# Download necessary NLTK data
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

# Correctly get the list of stopwords
stop_words = set(stopwords.words('english'))


try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')


from nltk import word_tokenize

# remove stopwords
def remove_stopwords(text):
    #splitting text to tokens
    tokens=nltk.tokenize.word_tokenize(text)

    filtered_tokens=[word for word in tokens if word not in stop_words]
    filtered_text=' '.join(filtered_tokens) # Join the filtered tokens back with a space
    return filtered_text

df['review']=df['review'].apply(remove_stopwords)
display(df.head())

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive


#
# Convert sentiment labels to integers

In [42]:
# Reload the dataframe to ensure the 'sentiment' column is still strings
df = pd.read_csv('data/IMDB Dataset.csv')

df['sentiment'] = df['sentiment'].str.strip().str.lower()

df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == "positive" else 0)

display(df.head())

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [43]:
df['sentiment'].unique()


array([1, 0])

In [44]:
print(df['sentiment'].value_counts())

sentiment
1    25000
0    25000
Name: count, dtype: int64


In [45]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# Remove Html Tags

In [46]:
import re
def remove_html(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

df['review']=df['review'].apply(remove_html)
df['review']


,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. The filming tec...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...
49995,I thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...


# Tokenization

In [47]:
from nltk import word_tokenize
df['review']=df['review'].apply(word_tokenize)
df.head()

,review,sentiment
0,"[One, of, the, other, reviewers, has, mentione...",1
1,"[A, wonderful, little, production, ., The, fil...",1
2,"[I, thought, this, was, a, wonderful, way, to,...",1
3,"[Basically, there, 's, a, family, where, a, li...",0
4,"[Petter, Mattei, 's, ``, Love, in, the, Time, ...",1


In [48]:
df['review'][0]

['One',
 'of',
 'the',
 'other',
 'reviewers',
 'has',
 'mentioned',
 'that',
 'after',
 'watching',
 'just',
 '1',
 'Oz',
 'episode',
 'you',
 "'ll",
 'be',
 'hooked',
 '.',
 'They',
 'are',
 'right',
 ',',
 'as',
 'this',
 'is',
 'exactly',
 'what',
 'happened',
 'with',
 'me.The',
 'first',
 'thing',
 'that',
 'struck',
 'me',
 'about',
 'Oz',
 'was',
 'its',
 'brutality',
 'and',
 'unflinching',
 'scenes',
 'of',
 'violence',
 ',',
 'which',
 'set',
 'in',
 'right',
 'from',
 'the',
 'word',
 'GO',
 '.',
 'Trust',
 'me',
 ',',
 'this',
 'is',
 'not',
 'a',
 'show',
 'for',
 'the',
 'faint',
 'hearted',
 'or',
 'timid',
 '.',
 'This',
 'show',
 'pulls',
 'no',
 'punches',
 'with',
 'regards',
 'to',
 'drugs',
 ',',
 'sex',
 'or',
 'violence',
 '.',
 'Its',
 'is',
 'hardcore',
 ',',
 'in',
 'the',
 'classic',
 'use',
 'of',
 'the',
 'word.It',
 'is',
 'called',
 'OZ',
 'as',
 'that',
 'is',
 'the',
 'nickname',
 'given',
 'to',
 'the',
 'Oswald',
 'Maximum',
 'Security',
 'State',
 '

# Split the train and test data

In [49]:
from sklearn.model_selection import train_test_split
X,y=df['review'].values,df['sentiment'].values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [50]:
X_train.shape,X_test.shape

((40000,), (10000,))

# Vocabulary Creation

In [52]:
from collections import Counter
all_tokens=[token for tokens in X_train for token in tokens]
word_counts=Counter(word_counts)
len(word_counts)

1671

### vocab size-2 becaue we already put 2 word 0 and 1 in unk and pad
### word2idx[word]=i+2 beacuse sequence start from 2 ans  0 and 1 and pad and unk

### most common word taking because most common will be the sentiment


In [57]:
vocab_size=2000
word2idx={"<PAD>":0,"<UNK>":0}
for i , (word,_) in enumerate (word_counts.most_common(vocab_size-2)):
  word2idx[word]=i+2
print(len(word2idx))

1673


In [61]:
#display(word2idx)

# Encoding +Padding

## Neural network cant understand text it understand number:words → encoding converts numbers which can be fed to nn.Embedding.

#### dictionary.get(key, default_value) here Looks for key in the dictionary.If key exists → returns dictionary[key].f key does not exist → returns default_value.

In [94]:
max_len=200
def encode_tokens(tokens):
  encoded=[word2idx.get(word,1) for word in tokens]

  if len(encoded) > max_len:
    encoded = encoded[:max_len]
  elif len(encoded) < max_len:
    encoded = encoded + [word2idx.get("<PAD>", 0)] * (max_len - len(encoded))

  return encoded

# Dataset and DataLoader

In [96]:
from torch.utils.data import Dataset, DataLoader

class IMDBDataset(Dataset):
  def __init__(self,tokens,labels):
    self.texts=[encode_tokens(t) for t in tokens]
    self.labels=labels

  def __len__(self):
    return len(self.labels)

  def __getitem__(self,idx):
    return torch.tensor(self.texts[idx]),torch.tensor(self.labels[idx])

In [97]:
train_dataset=IMDBDataset(X_train,y_train)
test_dataset=IMDBDataset(X_test,y_test)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [98]:
train_dataset[1]

(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(1))

In [99]:
len(train_dataset)

40000

# Build LSTM model

In [103]:
import torch.nn as nn

class SentimentLSTM(nn.Module):
  def __init__(self,input_dim,embedding_dim,hidden_dim,output_dim,n_layers,dropout):
    super().__init__()
    self.embedding=nn.Embedding(input_dim,embedding_dim)
    self.lstm=nn.LSTM(embedding_dim,hidden_dim,n_layers,dropout=dropout,batch_first=True)
    self.fc=nn.Linear(hidden_dim,output_dim)
    self.dropout=nn.Dropout(dropout)

  def forward(self, x):
    embedded=self.dropout(self.embedding(x))
    output,(hidden,cell)=self.lstm(embedded) # Use embedded instead of embedding
    hidden=self.dropout(hidden[-1])
    return self.fc(hidden)

In [105]:
input_dim=vocab_size
embedding_size=128
hidden_dim=256
output_dim=1
n_layers=2
dropout=0.5
model=SentimentLSTM(input_dim,embedding_size,hidden_dim,output_dim,n_layers,dropout)
model.to(device)


SentimentLSTM(
  (embedding): Embedding(2000, 128)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

# Training Setup
###  BCEWithLogitsLoss is Binary Cross Entropy with Logits Loss.Combines sigmoid activation + binary cross entropy loss in one step.Designed for binary classification tasks, e.g., sentiment analysis (positive vs negative).

In [106]:
import torch
import torch.nn as nn
import torch.optim as optim
epochs=20
learning_rate=0.001
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

# Trainign and Evaluation

In [107]:
def traning_model(model,loader,optimizer,criterion):
 model.train()
 total_loss=0

 for inputs,labels in loader:
  inputs,labels=inputs.to(device),labels.to(device)
  optimizer.zero_grad()
  output=model(inputs)
  loss=criterion(output.squeeze(),labels.float())
  loss.backward()
  optimizer.step()
  total_loss+=loss.item()
 return total_loss/len(loader)

# Evaluate

In [108]:
def evaluate_model(model,loader,criterion):
  model.eval()
  train_loss=0
  correct=0
  with torch.no_grad():
    for inputs,labels in loader:
      inputs,labels=inputs.to(device),labels.to(device)
      output=model(inputs)
      loss=criterion(output.squeeze(),labels.float())
      train_loss+=loss.item()
      predicted=torch.round(torch.sigmoid(output.squeeze()))
      correct+=(predicted==labels).sum().item()
  accuracy=correct/len(loader.dataset)
  return train_loss/len(loader), accuracy

train loop

In [109]:
for epoch in range(epochs):
  train_loss=traning_model(model,train_loader,optimizer,criterion)
  test_loss,accuracy=evaluate_model(model,test_loader,criterion)
  print(f"Epoch {epoch+1}/{epochs} | Train loss:{train_loss:.4f}")
  print(f"Epoch {epoch+1}/{epochs} | Test loss:{test_loss:.4f} | Accuracy:{accuracy:.4f}")

Epoch 1/20 | Train loss:0.6942
Epoch 1/20 | Test loss:0.6931 | Accuracy:0.5098
Epoch 2/20 | Train loss:0.6936
Epoch 2/20 | Test loss:0.6934 | Accuracy:0.5090
Epoch 3/20 | Train loss:0.6934
Epoch 3/20 | Test loss:0.6934 | Accuracy:0.5090
Epoch 4/20 | Train loss:0.6935
Epoch 4/20 | Test loss:0.6930 | Accuracy:0.5044
Epoch 5/20 | Train loss:0.6933
Epoch 5/20 | Test loss:0.6931 | Accuracy:0.5090
Epoch 6/20 | Train loss:0.6934
Epoch 6/20 | Test loss:0.6932 | Accuracy:0.4956
Epoch 7/20 | Train loss:0.6932
Epoch 7/20 | Test loss:0.6929 | Accuracy:0.5163
Epoch 8/20 | Train loss:0.6932
Epoch 8/20 | Test loss:0.6930 | Accuracy:0.4956
Epoch 9/20 | Train loss:0.6931
Epoch 9/20 | Test loss:0.6926 | Accuracy:0.5163
Epoch 10/20 | Train loss:0.6932
Epoch 10/20 | Test loss:0.6928 | Accuracy:0.4956
Epoch 11/20 | Train loss:0.6930
Epoch 11/20 | Test loss:0.6930 | Accuracy:0.5157
Epoch 12/20 | Train loss:0.6930
Epoch 12/20 | Test loss:0.6925 | Accuracy:0.5163
Epoch 13/20 | Train loss:0.6930
Epoch 13/20 | 

# Predict Function

In [116]:
def predict_sentment(model,text):
  model.eval()
  tokens=nltk.tokenize.word_tokenize(text)
  encoded_text=encode_tokens(tokens)
  input_tensor=torch.tensor(encoded_text).unsqueeze(0).to(device)
  with torch.no_grad():
    output=model(input_tensor)
    prob=torch.sigmoid(output).item()
  return "Positive" if prob>=0.5 else "Negative"

print(predict_sentment(model,"This movie was  good")
)

Positive
